In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
import csv

In [3]:
def scrape_reviews(product_url, num_pages):
  # List to store extracted review data
  reviews_data = []
  
  for page_num in range(1, num_pages + 1):
    # Construct the URL for the current page
    page_url = f"{product_url}&page={page_num}"
    
    # Send request and get HTML content
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all review elements on the page
    reviews = soup.find_all("...replace with selector for reviews...")  # Replace with appropriate CSS selector or HTML tag
  
    for review in reviews:
      # Extract reviewer name, date, and content using appropriate selectors
      reviewer_name = review.find("...replace with selector for reviewer name...").text.strip()  # Replace with appropriate selector
      review_date = review.find("...replace with selector for review date...").text.strip()  # Replace with appropriate selector
      review_content = review.find("...replace with selector for review content...").text.strip()  # Replace with appropriate selector
      
      # Add review data to the list
      reviews_data.append({
          "Reviewer Name": reviewer_name,
          "Review Date": review_date,
          "Review Content": review_content
      })

  # Write extracted data to CSV file
  with open("amazon_reviews.csv", "w", newline='') as csvfile:
    fieldnames = ["Reviewer Name", "Review Date", "Review Content"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(reviews_data)

# Replace with the actual product URL
product_url = "https://www.amazon.com/Philips-Sonicare-Rechargeable-Toothbrush-HX9990/dp/B08YCVJKXV/ref=sr_1_1_sspa?crid=38T1R3TDQJJHV&dib=eyJ2IjoiMSJ9.HYZaJyEUjdNmHlvNpEjYEVaDsILNrezMLofA0jak9fIvGnLysp3ooPqXQyW5U1JVcMCEd-tQe797MvmFmfaWTCQ5uIbz5XE0aLgg-lAm_Cltc6kAeMxxmOvCS5Paq3zwRZafsvh7ufdFo7iFIs7rThkYBlnYgR4doep

SyntaxError: unterminated string literal (detected at line 37) (210478397.py, line 37)

In [5]:
def scrape_reviews(product_url, num_pages):
  # List to store extracted review data
  reviews_data = []
  
  for page_num in range(1, num_pages + 1):
    # Construct the URL for the current page
    page_url = f"{product_url}&page={page_num}"
    
    # Send request and get HTML content
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all review elements on the page
    reviews = soup.find_all("...replace with selector for reviews...")  # Replace with appropriate CSS selector or HTML tag
  
    for review in reviews:
      # Extract reviewer name, date, and content using appropriate selectors
      reviewer_name = review.find("...replace with selector for reviewer name...").text.strip()  # Replace with appropriate selector
      review_date = review.find("...replace with selector for review date...").text.strip()  # Replace with appropriate selector
      review_content = review.find("...replace with selector for review content...").text.strip()  # Replace with appropriate selector
      
      # Add review data to the list
      reviews_data.append({
          "Reviewer Name": reviewer_name,
          "Review Date": review_date,
          "Review Content": review_content
      })

  # Write extracted data to CSV file
  with open("amazon_reviews.csv", "w", newline='') as csvfile:
    fieldnames = ["Reviewer Name", "Review Date", "Review Content"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(reviews_data)

# Replace with the actual product URL
product_url = "https://www.amazon.com/Philips-Sonicare-Rechargeable-Toothbrush-HX9990/dp/B08YCVJKXV"

In [6]:
def scrape_reviews(product_url, num_pages):
  # List to store extracted review data
  reviews_data = []
  
  for page_num in range(1, num_pages + 1):
    # Construct the URL for the current page
    page_url = f"{product_url}&page={page_num}"
    
    # Send request and get HTML content
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all review elements on the page
    reviews = soup.find_all("...replace with selector for reviews...")  # Replace with appropriate CSS selector or HTML tag
  
    for review in reviews:
      # Extract reviewer name, date, and content using appropriate selectors
      reviewer_name = review.find("...replace with selector for reviewer name...").text.strip()  # Replace with appropriate selector
      review_date = review.find("...replace with selector for review date...").text.strip()  # Replace with appropriate selector
      review_content = review.find("...replace with selector for review content...").text.strip()  # Replace with appropriate selector
      
      # Add review data to the list
      reviews_data.append({
          "Reviewer Name": reviewer_name,
          "Review Date": review_date,
          "Review Content": review_content
      })

  # Write extracted data to CSV file
  with open("amazon_reviews.csv", "w", newline='') as csvfile:
    fieldnames = ["Reviewer Name", "Review Date", "Review Content"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(reviews_data)

# Replace with the actual product URL
product_url = "https://www.amazon.com/Philips-Sonicare-Rechargeable-Toothbrush-HX9990/dp/B08YCVJKXV"

In [8]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_reviews(product_url, num_pages):
  # List to store extracted review data
  reviews_data = []
  
  for page_num in range(1, num_pages + 1):
    # Construct the URL for the current page
    page_url = f"{product_url}&page={page_num}"
    
    # Send request and get HTML content
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all review elements on the page - Replace with the appropriate selector for your target website
    reviews = soup.find_all("div", class = "a-profile-name")  # This might need adjustment for the target website

  for review in reviews:
      # Extract reviewer name, date, and content using appropriate selectors
      reviewer_name = review.find("span", itemprop="reviewerName").text.strip()  # Replace with appropriate selector
      review_date = review.find("span", itemprop="datePublished").text.strip()  # Replace with appropriate selector
      review_content = review.find("span", id="review-text-content").text.strip()  # Replace with appropriate selector
      
      # Add review data to the list
      reviews_data.append({
          "Reviewer Name": reviewer_name,
          "Review Date": review_date,
          "Review Content": review_content
      })

  # Write extracted data to CSV file
  with open("amazon_reviews.csv", "w", newline='') as csvfile:
    fieldnames = ["Reviewer Name", "Review Date", "Review Content"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(reviews_data)

# Replace with the actual product URL
product_url = "https://www.amazon.com/Philips-Sonicare-Rechargeable-Toothbrush-HX9990/dp/B08YCVJKXV"

scrape_reviews(product_url, 5)  # Change 5 to the desired number of pages (up to 5)


SyntaxError: invalid syntax (878457733.py, line 18)

In [13]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_reviews(product_url, num_pages):
  # List to store extracted review data
  reviews_data = []
  
  for page_num in range(1, num_pages + 1):
    # Construct the URL for the current page
    page_url = f"{product_url}&page={page_num}"
    
    # Send request and get HTML content
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all review elements on the page - Replace with the appropriate selector for your target website
    reviews = soup.find_all("div", class_= "a-profile-name")  # This might need adjustment for the target website

  for review in reviews:
      # Extract reviewer name, date, and content using appropriate selectors
      reviewer_name = review.find("span", itemprop="a-profile-name").text.strip()  # Replace with appropriate selector
      review_date = review.find("data-hook", itemprop="review-date").text.strip()  # Replace with appropriate selector
      review_content = review.find("span", id="a-letter-space").text.strip()  # Replace with appropriate selector
      
      # Add review data to the list
      reviews_data.append({
          "Reviewer Name": reviewer_name,
          "Review Date": review_date,
          "Review Content": review_content
      })

  # Write extracted data to CSV file
  with open("amazon_reviews.csv", "w", newline='') as csvfile:
    fieldnames = ["Reviewer Name", "Review Date", "Review Content"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(reviews_data)

# Replace with the actual product URL
product_url = "https://www.amazon.com/Philips-Sonicare-Rechargeable-Toothbrush-HX9990/product-reviews/B08YCVJKXV/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"

scrape_reviews(product_url, 5)  # Change 5 to the desired number of pages (up to 5)


In [14]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_reviews(product_url, num_pages):
  # List to store extracted review data
  reviews_data = []
  
  for page_num in range(1, num_pages + 1):
    # Construct the URL for the current page
    page_url = f"{product_url}&page={page_num}"
    
    # Send request and get HTML content
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all review elements on the page
    reviews = soup.find_all("div", class_="a-section a-spacing-medium")  # Adjust if needed for target website

    for review in reviews:
      # Extract reviewer name, date, and content using appropriate selectors
      reviewer_name = review.find("span", itemprop="reviewerName").text.strip()  # Replace if needed
      review_date = review.find("span", class_="a-size-base a-color-secondary review-date").text.strip()  # Updated selector for review date
      review_content = review.find("span", id="review-text-content").text.strip()  # Replace if needed
      
      # Add review data to the list
      reviews_data.append({
          "Reviewer Name": reviewer_name,
          "Review Date": review_date,
          "Review Content": review_content
      })

  # Write extracted data to CSV file
  with open("amazon_reviews.csv", "w", newline='') as csvfile:
    fieldnames = ["Reviewer Name", "Review Date", "Review Content"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(reviews_data)

# Replace with the actual product URL
product_url = "https://www.amazon.com/Philips-Sonicare-Rechargeable-Toothbrush-HX9990/product-reviews/B08YCVJKXV/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"

scrape_reviews(product_url, 5)  # Change 5 to the desired number of pages (up to 5)


In [20]:
def scrape_reviews(product_url, num_pages):
  # List to store extracted review data
  reviews_data = [id=="cm_cr-review_list"]
  
  for page_num in range(1, num_pages + 1):
    # Construct the URL for the current page
    page_url = f"{product_url}&page={page_num}"
    
    # Send request and get HTML content
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all review elements on the page
    reviews = soup.find_all("div", class_="a-section a-spacing-medium")  # Adjust if needed for target website

    for review in reviews:
      # Extract reviewer name, date, and content using appropriate selectors
      reviewer_name = review.find("span", class_="a-profile-name").text.strip()  # Replace if needed
      review_date = review.find("span", class_="a-size-base a-color-secondary review-date").text.strip()  # Updated selector for review date
      review_content = review.find("span", class_= "a-size-base review-text review-text-content").text.strip()  # Replace if needed
      
      # Add review data to the list
      reviews_data.append({
          "Reviewer Name": reviewer_name,
          "Review Date": review_date,
          "Review Content": review_content
      })

  # Write extracted data to CSV file
  with open("amazon_reviews.csv", "w", newline='') as csvfile:
    fieldnames = ["Reviewer Name", "Review Date", "Review Content"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(reviews_data)

# Replace with the actual product URL
product_url = "https://www.amazon.com/Philips-Sonicare-Rechargeable-Toothbrush-HX9990/product-reviews/B08YCVJKXV/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"

scrape_reviews(product_url, 5)  # Change 5 to the desired number of pages (up to 5)


AttributeError: 'bool' object has no attribute 'keys'

In [25]:
url = "https://www.amazon.com/Philips-Sonicare-Rechargeable-Toothbrush-HX9990/product-reviews/B08YCVJKXV/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"


def scrape_reviews(product_url, num_pages):
  # List to store extracted review data
  reviews_data = []
  
  for page_num in range(1, num_pages + 1):
    # Construct the URL for the current page
    page_url = f"{product_url}&page={page_num}"
    
    # Send request and get HTML content
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all review elements on the page
    reviews = soup.find_all("div", class_="a-section a-spacing-medium")  # Adjust if needed for target website

    for review in reviews:
      # Extract reviewer name, date, and content using appropriate selectors
      reviewer_name = review.find("span", class_="a-profile-name").text.strip()  # Updated selector for reviewer name
      review_date = review.find("span", class_="a-size-base a-color-secondary review-date").text.strip()  
      review_content = review.find("span", class_="a-size-base review-text review-text-content").text.strip() 
      
      # Add review data to the list
      reviews_data.append({
          "Reviewer Name": reviewer_name,
          "Review Date": review_date,
          "Review Content": review_content
      })

  # Write extracted data to CSV file
  with open("amazon_reviews.csv", "w", newline='') as csvfile:
    fieldnames = ["Name", "Date", "Content"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(reviews_data)

# Replace with the actual product URL
product_url = "https://www.amazon.com/Philips-Sonicare-Rechargeable-Toothbrush-HX9990/product-reviews/B08YCVJKXV/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"

scrape_reviews(product_url, 5)  # Change 5 to the desired number of pages (up to 5)

In [26]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_reviews(product_url, num_pages):
    reviews_data = []
    for page_num in range(1, num_pages + 1):
        page_url = f"{product_url}&page={page_num}"
        response = requests.get(page_url)  # Capture the response
        soup = BeautifulSoup(response.content, 'html.parser')
        reviews = soup.find_all("div", class_="a-section a-spacing-medium")
        for review in reviews:
            reviewer_name = review.find("span", class_="a-profile-name").text.strip()
            review_date = review.find("span", class_="a-size-base a-color-secondary review-date").text.strip()
            review_content = review.find("span", class_="a-size-base review-text review-text-content").text.strip()
            reviews_data.append({
                "Name": reviewer_name,
                "Date": review_date,
                "Content": review_content
            })
    with open("amazon_reviews.csv", "w", newline='') as csvfile:
        fieldnames = ["Name", "Date", "Content"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(reviews_data)

product_url = "https://www.amazon.com/Philips-Sonicare-Rechargeable-Toothbrush-HX9990/product-reviews/B08YCVJKXV/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"
scrape_reviews(product_url, 5)


In [27]:
import re
import json
import requests

url = 'https://shopee.com.my/Ins-Style-Niche-Exquisite-Flower-Graffiti-Cartoon-for-iPhone-Case-15-14-13-12-11-pro-max-7-8-plus-xr-x-xs-max-15pro-i.836359116.24714250751?sp_atk=b7958645-93b6-4107-a7a8-d3f7f23003e2&xptdk=b7958645-93b6-4107-a7a8-d3f7f23003e2'

r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

offset = 0
while True:
    ratings_url = f'https://shopee.com.my/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'
    data = requests.get(ratings_url).json()

    for rating in data['data']['ratings']:
        print(rating['author_username'])
        print(rating['comment'])
        print('-' * 80)

    if len(data['data']['ratings']) < 20:
        break

    offset += 20

s*****1
Material:good
Quality:goood

terima kasih barang sampai dlm keaddan baik mantap
--------------------------------------------------------------------------------
z*****m
Material:silicon ?? idkk😭
Quality:THICKKK, GOODDD

THIS CASING PUNYA COVERAGE PUN KINDA GOOODD DAHH LA CORAK COMEL CHEAP OULAK TUUUU GO GO BUY EMM
--------------------------------------------------------------------------------
zozopei42
Material:Matte silicon

The colour and material is superb. I’m so drawn into the quality. Definitely a good purchase
--------------------------------------------------------------------------------
s*****a
Quality:good

this is not for xr hm
--------------------------------------------------------------------------------
ufai88
Cantik sngat casing. Nnti nak beli lagi
--------------------------------------------------------------------------------
syipakkkkkk
Material:ok but not what i chose
--------------------------------------------------------------------------------
marhanif

beuhhhhhhh
Loveeee
--------------------------------------------------------------------------------
f*****1
Quality:ok
Material:ok

good
--------------------------------------------------------------------------------
melatildyz2

--------------------------------------------------------------------------------
manisasuffiya

--------------------------------------------------------------------------------
s*****k
Quality:GOOD
Material:PRETTY
--------------------------------------------------------------------------------
f*****_

--------------------------------------------------------------------------------
dieyna93

--------------------------------------------------------------------------------
dillahannah

--------------------------------------------------------------------------------
shfqhzmri

--------------------------------------------------------------------------------
batrisyanajib

--------------------------------------------------------------------------------
nursyafikaz

norhayati9906

--------------------------------------------------------------------------------
t4lfpwt4em

--------------------------------------------------------------------------------
karmibazilahkarim

--------------------------------------------------------------------------------
angellynnevanemarshavelii

--------------------------------------------------------------------------------
yeppahhhhh

--------------------------------------------------------------------------------
haisnksnams

--------------------------------------------------------------------------------
pinky9378

--------------------------------------------------------------------------------
ldynreyka

--------------------------------------------------------------------------------
nizamzekeria

--------------------------------------------------------------------------------
h*****h

--------------------------------------------------------------------------------
anirajak

-------------------------------------

nooreycah98

--------------------------------------------------------------------------------
norazlin02_

--------------------------------------------------------------------------------


In [28]:
import re
import json
import requests

url = 'https://shopee.com.my/venture.os/25654303483'  # Replace with the desired Shopee product URL

r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
  shop_id, item_id = r.group(1), r.group(2)
else:
  print("Invalid URL")
  exit()

offset = 0
while True:
  ratings_url = f'https://shopee.com.my/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'
  data = requests.get(ratings_url).json()

  for rating in data['data']['ratings']:
    print(rating['author_username'])
    print(rating['comment'])
    print('-' * 80)

  if len(data['data']['ratings']) < 20:
    break

  offset += 20

Invalid URL
s*****1
Material:good
Quality:goood

terima kasih barang sampai dlm keaddan baik mantap
--------------------------------------------------------------------------------
z*****m
Material:silicon ?? idkk😭
Quality:THICKKK, GOODDD

THIS CASING PUNYA COVERAGE PUN KINDA GOOODD DAHH LA CORAK COMEL CHEAP OULAK TUUUU GO GO BUY EMM
--------------------------------------------------------------------------------
zozopei42
Material:Matte silicon

The colour and material is superb. I’m so drawn into the quality. Definitely a good purchase
--------------------------------------------------------------------------------
s*****a
Quality:good

this is not for xr hm
--------------------------------------------------------------------------------
ufai88
Cantik sngat casing. Nnti nak beli lagi
--------------------------------------------------------------------------------
syipakkkkkk
Material:ok but not what i chose
-----------------------------------------------------------------------------

beuhhhhhhh
Loveeee
--------------------------------------------------------------------------------
f*****1
Quality:ok
Material:ok

good
--------------------------------------------------------------------------------
melatildyz2

--------------------------------------------------------------------------------
manisasuffiya

--------------------------------------------------------------------------------
s*****k
Quality:GOOD
Material:PRETTY
--------------------------------------------------------------------------------
f*****_

--------------------------------------------------------------------------------
dieyna93

--------------------------------------------------------------------------------
dillahannah

--------------------------------------------------------------------------------
shfqhzmri

--------------------------------------------------------------------------------
batrisyanajib

--------------------------------------------------------------------------------
nursyafikaz

norhayati9906

--------------------------------------------------------------------------------
t4lfpwt4em

--------------------------------------------------------------------------------
karmibazilahkarim

--------------------------------------------------------------------------------
angellynnevanemarshavelii

--------------------------------------------------------------------------------
yeppahhhhh

--------------------------------------------------------------------------------
haisnksnams

--------------------------------------------------------------------------------
pinky9378

--------------------------------------------------------------------------------
ldynreyka

--------------------------------------------------------------------------------
nizamzekeria

--------------------------------------------------------------------------------
h*****h

--------------------------------------------------------------------------------
anirajak

-------------------------------------

nooreycah98

--------------------------------------------------------------------------------
norazlin02_

--------------------------------------------------------------------------------


In [1]:
import re
import json
import requests
import time

url = 'https://shopee.com.my/304-Stainless-Steel-Camping-Water-Cup-Mug-Cawan-Air-Hiking-Outdoor-Camping-Cup-Mug-Set-i.1135285546.25654303483?xptdk=137097b9-7a41-4a14-9bb4-d0a831d7603c'  # Replace with the desired Shopee product URL

r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

offset = 0
while True:
    ratings_url = f'https://shopee.com.my/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'
    try:
        response = requests.get(ratings_url)
        response.raise_for_status()  # Raise an exception for 4XX and 5XX status codes
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        break

    for rating in data['data']['ratings']:
        print(rating['author_username'])
        print(rating['comment'])
        print('-' * 80)

    if len(data['data']['ratings']) < 20:
        break

    offset += 20
    time.sleep(1)  # Add a delay of 1 second between requests to avoid overwhelming the server


kamal6269
Quality:good
Functionality:good

Barang sampai dalam keadaan selamat.. cantik dan bergaya.. sesuai isi air sejuk klau air panas body semua panas..
--------------------------------------------------------------------------------
barib74
Functionality:mantap
Quality:mantap

Trusted seller...akan repeat..
--------------------------------------------------------------------------------
xindypui
304 stainless steel camping water cup mug
规格white,black,red
比我想象小
--------------------------------------------------------------------------------
jasminbalkis89
Functionality:Good
Quality:Good

Jenis steel tebal. 300ml kalau tak salah la..Size sedap2 pegang Just nice org kata. Silap nye mak tekan 1 item je..patut nak beli 2 item. Dah mmbazir delivery cost nk kena oder lagi 😅
--------------------------------------------------------------------------------
a*****n
Safely received my order with a good condition. Thank you seller.
--------------------------------------------------------------

In [2]:
import re
import json
import requests
import csv

url = 'https://shopee.com.my/304-Stainless-Steel-Camping-Water-Cup-Mug-Cawan-Air-Hiking-Outdoor-Camping-Cup-Mug-Set-i.1135285546.25654303483?xptdk=137097b9-7a41-4a14-9bb4-d0a831d7603c'  # Replace with the desired Shopee product URL

r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

offset = 0
ratings_data = []

while True:
    ratings_url = f'https://shopee.com.my/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'
    data = requests.get(ratings_url).json()

    for rating in data['data']['ratings']:
        author_username = rating['author_username']
        comment = rating['comment']
        ratings_data.append([author_username, comment])

    if len(data['data']['ratings']) < 20:
        break

    offset += 20

# Write data to CSV file
csv_file_name = 'ratings.csv'
with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Author Username', 'Comment'])  # Write header
    writer.writerows(ratings_data)

print(f"Data has been saved to {csv_file_name}.")


Data has been saved to ratings.csv.


In [4]:
import re
import json
import requests
import csv

url = 'https://shopee.com.my/304-Stainless-Steel-Camping-Water-Cup-Mug-Cawan-Air-Hiking-Outdoor-Camping-Cup-Mug-Set-i.1135285546.25654303483?xptdk=137097b9-7a41-4a14-9bb4-d0a831d7603c'  # Replace with the desired Shopee product URL

r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

offset = 0
ratings_data = []

while True:
    ratings_url = f'https://shopee.com.my/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'
    data = requests.get(ratings_url).json()

    for rating in data['data']['ratings']:
        author_username = rating['author_username']
        comment = rating['comment'].replace('\n', '  ')  # Replace newline characters with a space
        ratings_data.append([author_username, comment])

    if len(data['data']['ratings']) < 20:
        break

    offset += 20

# Write data to CSV file
csv_file_name = 'ratings.csv'
with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Author Username', 'Comment'])  # Write header
    writer.writerows(ratings_data)

print(f"Data has been saved to {csv_file_name}.")


Data has been saved to ratings.csv.


In [5]:
import re
import json
import requests
import csv
from bs4 import BeautifulSoup

url = 'https://shopee.com.my/304-Stainless-Steel-Camping-Water-Cup-Mug-Cawan-Air-Hiking-Outdoor-Camping-Cup-Mug-Set-i.1135285546.25654303483?xptdk=137097b9-7a41-4a14-9bb4-d0a831d7603c'  # Replace with the desired Shopee product URL

r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

offset = 0
ratings_data = []

while True:
    ratings_url = f'https://shopee.com.my/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'
    data = requests.get(ratings_url).json()

    for rating in data['data']['ratings']:
        author_username = rating['author_username']
        comment = rating['comment'].replace('\n', ' ')  # Replace newline characters with a space
        
        # Extracting date information from the review time div element
        review_time = rating['comment_info']
        soup = BeautifulSoup(review_time, 'html.parser')
        review_date = soup.find('div', class_='shopee-product-rating__time').text.strip()
        
        ratings_data.append([author_username, comment, review_date])

    if len(data['data']['ratings']) < 20:
        break

    offset += 20

# Write data to CSV file
csv_file_name = 'ratings.csv'
with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Author Username', 'Comment', 'Review Date'])  # Write header
    writer.writerows(ratings_data)

print(f"Data has been saved to {csv_file_name}.")


KeyError: 'comment_info'

In [10]:
import re
import requests
import csv
from bs4 import BeautifulSoup

url = 'https://shopee.com.my/304-Stainless-Steel-Camping-Water-Cup-Mug-Cawan-Air-Hiking-Outdoor-Camping-Cup-Mug-Set-i.1135285546.25654303483?xptdk=137097b9-7a41-4a14-9bb4-d0a831d7603c'  # Replace with the desired Shopee product URL

r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

offset = 0
ratings_data = []

while True:
    ratings_url = f'https://shopee.com.my/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'
    data = requests.get(ratings_url).json()

    for rating in data['data']['ratings']:
        author_username = rating['author_username']
        comment = rating['comment'].replace('\n', ' ')  # Replace newline characters with a space
        
        # Extracting date information from the comment
        soup = BeautifulSoup(comment, 'html.parser')
        review_date = soup.find('div', class_='shopee-product-rating__main').text.strip()
        
        ratings_data.append([author_username, comment, review_date])

    if len(data['data']['ratings']) < 20:
        break

    offset += 20

# Write data to CSV file
csv_file_name = 'ratings.csv'
with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Author Username', 'Comment', 'Review Date'])  # Write header
    writer.writerows(ratings_data)

print(f"Data has been saved to {csv_file_name}.")


AttributeError: 'NoneType' object has no attribute 'text'

In [11]:
import re
import requests
import csv

url = 'https://shopee.com.my/304-Stainless-Steel-Camping-Water-Cup-Mug-Cawan-Air-Hiking-Outdoor-Camping-Cup-Mug-Set-i.1135285546.25654303483?xptdk=137097b9-7a41-4a14-9bb4-d0a831d7603c'  # Replace with the desired Shopee product URL

r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

offset = 0
ratings_data = []

while True:
    ratings_url = f'https://shopee.com.my/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'
    data = requests.get(ratings_url).json()

    for rating in data['data']['ratings']:
        author_username = rating['author_username']
        comment = rating['comment'].replace('\n', ' ')  # Replace newline characters with a space
        
        # Extracting date information from the comment using regular expressions
        match = re.search(r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2})', comment)
        if match:
            review_date = match.group(1)
        else:
            review_date = "Date not found"

        ratings_data.append([author_username, comment, review_date])

    if len(data['data']['ratings']) < 20:
        break

    offset += 20

# Write data to CSV file
csv_file_name = 'ratings.csv'
with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Author Username', 'Comment', 'Review Date'])  # Write header
    writer.writerows(ratings_data)

print(f"Data has been saved to {csv_file_name}.")


Data has been saved to ratings.csv.


In [ ]:
import re
import json
import requests
import csv

url = 'https://shopee.com.my/JISULIFE-3-in-1-Portable-Fan-9000-mAh-Battery-Rechargeable-With-Air-Duster-Function-Mini-Fan-With-1-100-Speeds-LED-Display-Outdoor-Camping-i.310060557.23154647628'  # Replace with the desired Shopee product URL

r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

offset = 0
ratings_data = []

while True:
    ratings_url = f'https://shopee.com.my/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'
    data = requests.get(ratings_url).json()

    for rating in data['data']['ratings']:
        author_username = rating['author_username']
        comment = rating['comment'].replace('\n', ' ')  # Replace newline characters with a space
        
        # Extracting date information from the API response
        review_date = rating['ctime']

        ratings_data.append([author_username, comment, review_date])

    if len(data['data']['ratings']) < 20:
        break

    offset += 20

# Write data to CSV file
csv_file_name = 'ratings.csv'
with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Author Username', 'Comment', 'Review Date'])  # Write header
    writer.writerows(ratings_data)

print(f"Data has been saved to {csv_file_name}.")


In [ ]:
import re
import json
import requests
import csv

url = 'https://shopee.com.my/JISULIFE-3-in-1-Portable-Fan-9000-mAh-Battery-Rechargeable-With-Air-Duster-Function-Mini-Fan-With-1-100-Speeds-LED-Display-Outdoor-Camping-i.310060557.23154647628'  # Replace with the desired Shopee product URL

r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

offset = 0
ratings_data = []

while True:
    ratings_url = f'https://shopee.com.my/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'
    data = requests.get(ratings_url).json()

    for rating in data['data']['ratings']:
        author_username = rating['author_username']
        comment = rating['comment'].replace('\n', ' ')  # Replace newline characters with a space
        
        # Extracting date information from the API response
        review_date = rating['ctime']

        ratings_data.append([author_username, comment, review_date])

    if len(data['data']['ratings']) < 20:
        break

    offset += 20

# Write data to CSV file
csv_file_name = 'ratings.csv'
with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Author Username', 'Comment', 'Review Date'])  # Write header
    writer.writerows(ratings_data)

print(f"Data has been saved to {csv_file_name}.")


In [ ]:
import re
import json
import requests
import csv

url = 'https://shopee.com.my/JISULIFE-3-in-1-Portable-Fan-9000-mAh-Battery-Rechargeable-With-Air-Duster-Function-Mini-Fan-With-1-100-Speeds-LED-Display-Outdoor-Camping-i.310060557.23154647628'  # Replace with the desired Shopee product URL

r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

offset = 0
ratings_data = []

while True:
    ratings_url = f'https://shopee.com.my/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'
    data = requests.get(ratings_url).json()

    for rating in data['data']['ratings']:
        author_username = rating['author_username']
        comment = rating['comment'].replace('\n', ' ')  # Replace newline characters with a space
        
        # Extracting date information from the API response
        review_date = rating['ctime']

        ratings_data.append([author_username, comment, review_date])

    if len(data['data']['ratings']) < 20:
        break

    offset += 20

# Write data to CSV file
csv_file_name = 'ratings.csv'
with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Author Username', 'Comment', 'Review Date'])  # Write header
    writer.writerows(ratings_data)

print(f"Data has been saved to {csv_file_name}.")


In [ ]:
import re
import json
import requests
import csv

url = 'https://shopee.com.my/JISULIFE-3-in-1-Portable-Fan-9000-mAh-Battery-Rechargeable-With-Air-Duster-Function-Mini-Fan-With-1-100-Speeds-LED-Display-Outdoor-Camping-i.310060557.23154647628'  # Replace with the desired Shopee product URL

r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

offset = 0
ratings_data = []

while True:
    ratings_url = f'https://shopee.com.my/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'
    data = requests.get(ratings_url).json()

    for rating in data['data']['ratings']:
        author_username = rating['author_username']
        comment = rating['comment'].replace('\n', '  ')  # Replace newline characters with a space
        ratings_data.append([author_username, comment])

    if len(data['data']['ratings']) < 20:
        break

    offset += 20

# Write data to CSV file
csv_file_name = 'ratings.csv'
with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Author Username', 'Comment'])  # Write header
    writer.writerows(ratings_data)

print(f"Data has been saved to {csv_file_name}.")

In [ ]:
import re
import json
import requests
import csv

In [2]:
url = 'https://shopee.com.my/JISULIFE-3-in-1-Portable-Fan-9000-mAh-Battery-Rechargeable-With-Air-Duster-Function-Mini-Fan-With-1-100-Speeds-LED-Display-Outdoor-Camping-i.310060557.23154647628'  # Replace with the desired Shopee product URL

r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

offset = 0
ratings_data = []

while True:
    ratings_url = f'https://shopee.com.my/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'
    data = requests.get(ratings_url).json()

    for rating in data['data']['ratings']:
        author_username = rating['author_username']
        comment = rating['comment'].replace('\n', '  ')  # Replace newline characters with a space
        ratings_data.append([author_username, comment])

    if len(data['data']['ratings']) < 20:
        break

    offset += 20

# Write data to CSV file
csv_file_name = 'ratings.csv'
with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Author Username', 'Comment'])  # Write header
    writer.writerows(ratings_data)

print(f"Data has been saved to {csv_file_name}.")

TypeError: 'NoneType' object is not iterable

In [3]:
import re
import json
import requests
import csv

url = 'https://shopee.com.my/JISULIFE-3-in-1-Portable-Fan-9000-mAh-Battery-Rechargeable-With-Air-Duster-Function-Mini-Fan-With-1-100-Speeds-LED-Display-Outdoor-Camping-i.310060557.23154647628'  # Replace with the desired Shopee product URL

r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

offset = 0
ratings_data = []

while True:
    ratings_url = f'https://shopee.com.my/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'
    response = requests.get(ratings_url)

    if response.status_code == 200:
        data = response.json()
        if 'data' in data and 'ratings' in data['data']:
            for rating in data['data']['ratings']:
                author_username = rating['author_username']
                comment = rating['comment'].replace('\n', '  ')  # Replace newline characters with a space
                ratings_data.append([author_username, comment])
                
            if len(data['data']['ratings']) < 20:
                break
                
            offset += 20
        else:
            print("Error: Unexpected response format. Unable to retrieve ratings data.")
            exit()
    else:
        print(f"Error: Failed to fetch data from the API. Status code: {response.status_code}")
        exit()

# Write data to CSV file
csv_file_name = 'ratings.csv'
with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Author Username', 'Comment'])  # Write header
    writer.writerows(ratings_data)

print(f"Data has been saved to {csv_file_name}.")


TypeError: 'NoneType' object is not iterable

In [4]:
import re
import json
import requests
import time

url = 'https://shopee.com.my/JISULIFE-3-in-1-Portable-Fan-9000-mAh-Battery-Rechargeable-With-Air-Duster-Function-Mini-Fan-With-1-100-Speeds-LED-Display-Outdoor-Camping-i.310060557.23154647628'  # Replace with the desired Shopee product URL

r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

offset = 0
while True:
    ratings_url = f'https://shopee.com.my/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'
    try:
        response = requests.get(ratings_url)
        response.raise_for_status()  # Raise an exception for 4XX and 5XX status codes
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        break

    for rating in data['data']['ratings']:
        print(rating['author_username'])
        print(rating['comment'])
        print('-' * 80)

    if len(data['data']['ratings']) < 20:
        break

    offset += 20
    time.sleep(1)  # Add a delay of 1 second between requests to avoid overwhelming the server


h*****i
Portability:bagus, compact. Size kecik yang senang untuk dipegang
Performance:tahan berjam

Produk ni sangat bagus berbanding kipas angin di luar sana. Bateri pon tahan berjam. Saya pkai dalam masa 2jam bru habis 11% bateri. Guna speed 20sahaja. Tapi kalau pakai speed lagi kuat, makin cepat bateri habis. Untuk cuaca malaysia yang panas sekarang ni, pembelian yang sangat berbaloi. Bentuk size kecik. Comfortable untuk digenggam & berat sikit.
--------------------------------------------------------------------------------
jonathanlamkengyeen
Portability:easy to carry and also great build design - its a modern device for a handheld fan which can double as a mini fan (with stand) / hair dryer / dust blower
Performance:strong wind but need abit of distance to feel the gust of wind

A good product that seems to be build with good material and the wind generated is stronger than any of those mini plastic fan
-----------------------------------------------------------------------------

k*****9
Nice packing and fast delivery 👍👍👍 reasonable price, recommend to all.
--------------------------------------------------------------------------------
mdazizi88
Portability:ok
Performance:ok

penghantaran sgt pantas & produk sgt baik dan boleh digunakan
--------------------------------------------------------------------------------
z*****a
Performance:Great
Portability:The best👍🏻

Well packed & received in good condition. Bought it as a surprise gift for our son. 😊
--------------------------------------------------------------------------------
zaidimendes
Portability:mudah nak bawa dan simpan dalam beg
Performance:angin dia padu plus tak berat sangat

So far aku dah pakai dalam berapa hari ni, angin dia kuat macam kipas angin biasa cuma kelemahan dia. Kalau buka angin kuat, battery pun cepat drain hahaha so nasihat saya buka setakat 50 tu dah okey dah
--------------------------------------------------------------------------------
f*****6
Thanks for the best product. Easy to

eltonvaz
Performance:Fan is strong but quality bad
--------------------------------------------------------------------------------
jules72
This product is so cool!
--------------------------------------------------------------------------------
alamiru79
Performance:x power sangat
Portability:biasa je
--------------------------------------------------------------------------------
muttakin.jhon
Barang terbaik kipas sgt laju
--------------------------------------------------------------------------------
s*****m
Portability:good
Performance:ok
--------------------------------------------------------------------------------
monsterloi

--------------------------------------------------------------------------------
aimanfazil89
Okok
--------------------------------------------------------------------------------
hishamnaim93

--------------------------------------------------------------------------------
dixiontse

-----------------------------------------------------------------------

nadza5110
Good Item
--------------------------------------------------------------------------------
nadza5110
Good Item
--------------------------------------------------------------------------------
cavyn79
Very good
--------------------------------------------------------------------------------
f*****x
Portability:excellent
Performance:huricane
--------------------------------------------------------------------------------
baim491
Ok
--------------------------------------------------------------------------------
s*****a
Ok
--------------------------------------------------------------------------------
abu101099
gooood
--------------------------------------------------------------------------------
k*****9
Memang kuat angin die.
--------------------------------------------------------------------------------
aidiehaser
Performance:good
Portability:good good
--------------------------------------------------------------------------------
nuremalia92
Penghantaran laju sgt2..terbai

s*****g

--------------------------------------------------------------------------------
mohdimihamdan

--------------------------------------------------------------------------------
mnsyafiqhusen

--------------------------------------------------------------------------------
greyxv

--------------------------------------------------------------------------------
azri0610

--------------------------------------------------------------------------------
razcher

--------------------------------------------------------------------------------
adamqamarol

--------------------------------------------------------------------------------
jerryfish_enterprise

--------------------------------------------------------------------------------
ansorirazak

--------------------------------------------------------------------------------
nurulorlando

--------------------------------------------------------------------------------
r*****l

-----------------------------------------------------

...

m*****t

--------------------------------------------------------------------------------
sritatpun

--------------------------------------------------------------------------------
tunkang17

--------------------------------------------------------------------------------
sittichokkulsiri

--------------------------------------------------------------------------------
pop23

--------------------------------------------------------------------------------
peach_peach.peach

--------------------------------------------------------------------------------
birdzaji

--------------------------------------------------------------------------------
nine91011

--------------------------------------------------------------------------------
b*****s

--------------------------------------------------------------------------------
pech2532

--------------------------------------------------------------------------------
hrvhappy

-----------------------------------------------------------------

r*****w

--------------------------------------------------------------------------------
faquidiot

--------------------------------------------------------------------------------
suphakitzzaun

--------------------------------------------------------------------------------
mabi1221

--------------------------------------------------------------------------------
p*****p

--------------------------------------------------------------------------------
a*****p

--------------------------------------------------------------------------------
r*****n

--------------------------------------------------------------------------------
s*****i

--------------------------------------------------------------------------------
hs6ruk

--------------------------------------------------------------------------------
winyuleelasitorn

--------------------------------------------------------------------------------
m*****t

--------------------------------------------------------------------------

pockythedark

--------------------------------------------------------------------------------
j*****a

--------------------------------------------------------------------------------
wirotetim

--------------------------------------------------------------------------------
vkm.list

--------------------------------------------------------------------------------
floetwiratar

--------------------------------------------------------------------------------
nina_nina_16

--------------------------------------------------------------------------------
soch.popkrub

--------------------------------------------------------------------------------
tonton99999

--------------------------------------------------------------------------------
natat.c

--------------------------------------------------------------------------------
poowadee

--------------------------------------------------------------------------------
mootalwichu

-----------------------------------------------------------

m*****i

--------------------------------------------------------------------------------
y*****j

--------------------------------------------------------------------------------
foursix6666

--------------------------------------------------------------------------------
p*****w

--------------------------------------------------------------------------------
kittnof

--------------------------------------------------------------------------------
chutchawanem

--------------------------------------------------------------------------------
naweea

--------------------------------------------------------------------------------
p*****p

--------------------------------------------------------------------------------
akiratori

--------------------------------------------------------------------------------
thanchon

--------------------------------------------------------------------------------
sanuxkyaemsuk

--------------------------------------------------------------------------

TypeError: 'NoneType' object is not iterable

In [5]:
import re
import json
import requests
import csv

url = 'https://shopee.com.my/JISULIFE-3-in-1-Portable-Fan-9000-mAh-Battery-Rechargeable-With-Air-Duster-Function-Mini-Fan-With-1-100-Speeds-LED-Display-Outdoor-Camping-i.310060557.23154647628'  # Replace with the desired Shopee product URL

r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

offset = 0
ratings_data = []

while True:
    ratings_url = f'https://shopee.com.my/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'
    data = requests.get(ratings_url).json()

    for rating in data['data']['ratings']:
        author_username = rating['author_username']
        comment = rating['comment']
        ratings_data.append([author_username, comment])

    if len(data['data']['ratings']) < 20:
        break

    offset += 20

# Write data to CSV file
csv_file_name = 'ratings.csv'
with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Author Username', 'Comment'])  # Write header
    writer.writerows(ratings_data)

print(f"Data has been saved to {csv_file_name}.")


TypeError: 'NoneType' object is not iterable

In [ ]:
import re
import json
import requests
import csv

url = 'https://shopee.com.my/304-Stainless-Steel-Camping-Water-Cup-Mug-Cawan-Air-Hiking-Outdoor-Camping-Cup-Mug-Set-i.1135285546.25654303483?xptdk=137097b9-7a41-4a14-9bb4-d0a831d7603c'  # Replace with the desired Shopee product URL

r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

limit = 30  # Limit to 30 ratings
offset = 0
ratings_data = []

while True:
    ratings_url = f'https://shopee.com.my/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit={limit}&offset={offset}&shopid={shop_id}&type=0'
    data = requests.get(ratings_url).json()

    for rating in data['data']['ratings']:
        author_username = rating['author_username']
        comment = rating['comment']
        ratings_data.append([author_username, comment])

    if len(data['data']['ratings']) < limit or len(ratings_data) >= 30:  # Break if less than limit is returned or 30 ratings are collected
        break

    offset += limit

# Write data to CSV file
csv_file_name = 'ratings.csv'
with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Author Username', 'Comment'])  # Write header
    writer.writerows(ratings_data)

print(f"Data has been saved to {csv_file_name}.")


In [1]:
import re
import json
import requests
import csv
import datetime

# Replace the URL with the desired Shopee product URL
url = 'https://shopee.com.my/JISULIFE-3-in-1-Portable-Fan-9000-mAh-Battery-Rechargeable-With-Air-Duster-Function-Mini-Fan-With-1-100-Speeds-LED-Display-Outdoor-Camping-i.310060557.23154647628'

# Extract shop_id and item_id from the URL
r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

# Initialize offset and ratings_data list
offset = 0
ratings_data = []

# Loop to fetch ratings data, limited to 150 records
while offset < 150:
    # Construct ratings URL based on the provided documentation with a limit of 20 records per request
    ratings_url = f'https://shopee.sg/api/v2/item/get_ratings?shopid={shop_id}&itemid={item_id}&limit=20&offset={offset}&type=0&filter=0'

    # Fetch data from the API endpoint
    data = requests.get(ratings_url).json()

    # Extract relevant information from the API response
    for rating in data['data']['ratings']:
        author_username = rating['author_username']
        comment = rating['comment'].replace('\n', ' ')  # Replace newline characters with a space
        timestamp = rating['ctime']  # Extracting date information from the API response

        # Convert Unix timestamp to datetime object
        review_date = datetime.datetime.utcfromtimestamp(timestamp)
        # Format the datetime object as a string in a desired format
        formatted_review_date = review_date.strftime('%Y-%m-%d %H:%M:%S')

        ratings_data.append([author_username, comment, formatted_review_date])  # Append formatted review date

    # Break the loop if all desired records have been fetched
    if len(data['data']['ratings']) < 20:
        break

    offset += 20

# Write data to CSV file
csv_file_name = 'ratings.csv'
with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Author Username', 'Comment', 'Review Date'])  # Write header
    writer.writerows(ratings_data)

print(f"Data has been saved to {csv_file_name}.")


Data has been saved to ratings.csv.


In [2]:
import re
import json
import requests
import csv
import datetime

# Replace the URL with the desired Shopee product URL
url = 'https://shopee.com.my/JISULIFE-3-in-1-Portable-Fan-9000-mAh-Battery-Rechargeable-With-Air-Duster-Function-Mini-Fan-With-1-100-Speeds-LED-Display-Outdoor-Camping-i.310060557.23154647628'

# Extract shop_id and item_id from the URL
r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

# Initialize offset and ratings_data list
offset = 0
ratings_data = []

# Loop to fetch ratings data, limited to 150 records
while offset < 150:
    # Construct ratings URL based on the provided documentation with a limit of 20 records per request
    ratings_url = f'https://shopee.sg/api/v2/item/get_ratings?shopid={shop_id}&itemid={item_id}&limit=20&offset={offset}&type=0&filter=0'

    # Fetch data from the API endpoint
    data = requests.get(ratings_url).json()

    # Extract relevant information from the API response
    for rating in data['data']['ratings']:
        author_username = rating['author_username']
        comment = rating['comment'].replace('\n', ' ')  # Replace newline characters with a space
        timestamp = rating['ctime']  # Extracting date information from the API response

        # Convert Unix timestamp to datetime object
        review_date = datetime.datetime.utcfromtimestamp(timestamp)
        # Format the datetime object as a string with abbreviated month name
        formatted_review_date = review_date.strftime('%d-%b-%Y %H:%M:%S')

        ratings_data.append([author_username, comment, formatted_review_date])  # Append formatted review date

    # Break the loop if all desired records have been fetched
    if len(data['data']['ratings']) < 20:
        break

    offset += 20

# Write data to CSV file
csv_file_name = 'ratings.csv'
with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Author Username', 'Comment', 'Review Date'])  # Write header
    writer.writerows(ratings_data)

print(f"Data has been saved to {csv_file_name}.")


Data has been saved to ratings.csv.


In [3]:
import re
import json
import requests
import csv
import datetime

# Replace the URL with the desired Shopee product URL
url = 'https://shopee.com.my/JISULIFE-3-in-1-Portable-Fan-9000-mAh-Battery-Rechargeable-With-Air-Duster-Function-Mini-Fan-With-1-100-Speeds-LED-Display-Outdoor-Camping-i.310060557.23154647628'

# Extract shop_id and item_id from the URL
r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

# Initialize offset and ratings_data list
offset = 0
ratings_data = []

# Loop to fetch ratings data, limited to 150 records
while offset < 150:
    # Construct ratings URL based on the provided documentation with a limit of 20 records per request
    ratings_url = f'https://shopee.sg/api/v2/item/get_ratings?shopid={shop_id}&itemid={item_id}&limit=20&offset={offset}&type=0&filter=0'

    # Fetch data from the API endpoint
    data = requests.get(ratings_url).json()

    # Extract relevant information from the API response
    for rating in data['data']['ratings']:
        author_username = rating['author_username']
        comment = rating['comment'].replace('\n', ' ')  # Replace newline characters with a space
        timestamp = rating['ctime']  # Extracting date information from the API response

        # Convert Unix timestamp to datetime object
        review_date = datetime.datetime.utcfromtimestamp(timestamp)
        # Format the datetime object as a string with abbreviated month name
        formatted_review_date = review_date.strftime('%d-%b-%Y %H:%M:%S')

        ratings_data.append([author_username, comment, formatted_review_date])  # Append formatted review date

    # Break the loop if all desired records have been fetched
    if len(data['data']['ratings']) < 20:
        break

    offset += 20

# Write data to CSV file
csv_file_name = 'ratings.csv'
with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Author Username', 'Comment', 'Review Date'])  # Write header
    writer.writerows(ratings_data)

print(f"Data has been saved to {csv_file_name}.")


Data has been saved to ratings.csv.


In [4]:
import re
import json
import requests
import csv
import datetime

# Replace the URL with the desired Shopee product URL
url = 'https://shopee.com.my/JISULIFE-3-in-1-Portable-Fan-9000-mAh-Battery-Rechargeable-With-Air-Duster-Function-Mini-Fan-With-1-100-Speeds-LED-Display-Outdoor-Camping-i.310060557.23154647628'

# Extract shop_id and item_id from the URL
r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

# Initialize offset and ratings_data list
offset = 0
ratings_data = []

# Loop to fetch ratings data, limited to 150 records
while offset < 150:
    # Construct ratings URL based on the provided documentation with a limit of 20 records per request
    ratings_url = f'https://shopee.sg/api/v2/item/get_ratings?shopid={shop_id}&itemid={item_id}&limit=20&offset={offset}&type=0&filter=0'

    # Fetch data from the API endpoint
    data = requests.get(ratings_url).json()

    # Extract relevant information from the API response
    for rating in data['data']['ratings']:
        author_username = rating['author_username']
        comment = rating['comment'].replace('\n', ' ')  # Replace newline characters with a space
        timestamp = rating['ctime']  # Extracting date information from the API response

        # Convert Unix timestamp to datetime object
        review_date = datetime.datetime.utcfromtimestamp(timestamp)
        # Format the datetime object as a string with the desired format '1-Jan-2024 HH:MM:SS'
        formatted_review_date = review_date.strftime('%d-%b-%Y %H:%M:%S')

        ratings_data.append([author_username, comment, formatted_review_date])  # Append formatted review date

    # Break the loop if all desired records have been fetched
    if len(data['data']['ratings']) < 20:
        break

    offset += 20

# Write data to CSV file
csv_file_name = 'ratings.csv'
with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Author Username', 'Comment', 'Review Date'])  # Write header
    writer.writerows(ratings_data)

print(f"Data has been saved to {csv_file_name}.")


Data has been saved to ratings.csv.


In [5]:
import re
import json
import requests
import csv
import datetime

# Replace the URL with the desired Shopee product URL
url = 'https://shopee.com.my/JISULIFE-3-in-1-Portable-Fan-9000-mAh-Battery-Rechargeable-With-Air-Duster-Function-Mini-Fan-With-1-100-Speeds-LED-Display-Outdoor-Camping-i.310060557.23154647628'

# Extract shop_id and item_id from the URL
r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

# Initialize offset and ratings_data list
offset = 0
ratings_data = []

# Loop to fetch ratings data, limited to 150 records
while offset < 150:
    # Construct ratings URL based on the provided documentation with a limit of 20 records per request
    ratings_url = f'https://shopee.sg/api/v2/item/get_ratings?shopid={shop_id}&itemid={item_id}&limit=20&offset={offset}&type=0&filter=0'

    # Fetch data from the API endpoint
    data = requests.get(ratings_url).json()

    # Extract relevant information from the API response
    for rating in data['data']['ratings']:
        author_username = rating['author_username']
        comment = rating['comment'].replace('\n', ' ')  # Replace newline characters with a space
        timestamp = rating['ctime']  # Extracting date information from the API response

        # Convert Unix timestamp to datetime object
        review_date = datetime.datetime.utcfromtimestamp(timestamp)
        # Format the datetime object as a string with abbreviated month name
        formatted_review_date = review_date.strftime('%Y-%b-%d %H:%M:%S')

        ratings_data.append([author_username, comment, formatted_review_date])  # Append formatted review date

    # Break the loop if all desired records have been fetched
    if len(data['data']['ratings']) < 20:
        break

    offset += 20

# Write data to CSV file
csv_file_name = 'ratings.csv'
with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Author Username', 'Comment', 'Review Date'])  # Write header
    writer.writerows(ratings_data)

print(f"Data has been saved to {csv_file_name}.")


PermissionError: [Errno 13] Permission denied: 'ratings.csv'

In [6]:
import re
import json
import requests
import csv
import datetime

# Replace the URL with the desired Shopee product URL
url = 'https://shopee.com.my/JISULIFE-3-in-1-Portable-Fan-9000-mAh-Battery-Rechargeable-With-Air-Duster-Function-Mini-Fan-With-1-100-Speeds-LED-Display-Outdoor-Camping-i.310060557.23154647628'

# Extract shop_id and item_id from the URL
r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

# Initialize offset and ratings_data list
offset = 0
ratings_data = []

# Loop to fetch ratings data, limited to 150 records
while offset < 150:
    # Construct ratings URL based on the provided documentation with a limit of 20 records per request
    ratings_url = f'https://shopee.sg/api/v2/item/get_ratings?shopid={shop_id}&itemid={item_id}&limit=20&offset={offset}&type=0&filter=0'

    # Fetch data from the API endpoint
    data = requests.get(ratings_url).json()

    # Extract relevant information from the API response
    for rating in data['data']['ratings']:
        author_username = rating['author_username']
        comment = rating['comment'].replace('\n', ' ')  # Replace newline characters with a space
        timestamp = rating['ctime']  # Extracting date information from the API response

        # Convert Unix timestamp to datetime object
        review_date = datetime.datetime.utcfromtimestamp(timestamp)
        # Format the datetime object as a string with abbreviated month name
        formatted_review_date = review_date.strftime('%Y-%b-%d %H:%M:%S')

        ratings_data.append([author_username, comment, formatted_review_date])  # Append formatted review date

    # Break the loop if all desired records have been fetched
    if len(data['data']['ratings']) < 20:
        break

    offset += 20

# Write data to CSV file
csv_file_name = 'ratings.csv'
with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Author Username', 'Comment', 'Review Date'])  # Write header
    writer.writerows(ratings_data)

print(f"Data has been saved to {csv_file_name}.")


Data has been saved to ratings.csv.
